### 전체 목록 출력

In [13]:
import pandas as pd
import requests

API_KEY = '17db1c341b5874ad5ad8a43382af6b17d3009380'
URL = 'https://opendart.fss.or.kr/api/list.json'

params = {
    'crtfc_key': API_KEY,
    'pblntf_detail_ty': 'C004',
    'bgn_de': '20240401',
    'end_de': '20240630',
    'last_reprt_at': 'Y',
    'page_no': 1,
    'page_count': 100
}

resp = requests.get(URL, params=params)
resp.raise_for_status()
data = resp.json()
df = pd.DataFrame(data['list'])[['rcept_dt', 'corp_code', 'corp_name', 'corp_cls', 'report_nm', 'rcept_no', 'rm']].sort_values(by='rcept_dt', ascending=True).reset_index(drop=True)
df

,rcept_dt,corp_code,corp_name,corp_cls,report_nm,rcept_no,rm
0,20240405,01344363,다원넥스뷰,K,투자설명서,20240405000062,
1,20240411,01283974,삐아,K,증권발행실적보고서(합병등),20240411000658,
2,20240412,01429220,피아이이,K,철회신고서,20240412003248,
3,20240412,01601222,씨피시스템,K,[기재정정]증권신고서(합병),20240412000371,
4,20240419,01601222,씨피시스템,K,투자설명서,20240419000120,
5,20240419,01116265,카티스,K,증권발행실적보고서(합병등),20240419000061,
6,20240502,00306454,한국정보인증,K,증권발행실적보고서(합병등),20240502000361,
7,20240528,01344363,다원넥스뷰,K,[기재정정]증권발행실적보고서(합병등),20240528000294,
8,20240531,00117188,효성,Y,[기재정정]증권신고서(분할),20240531000956,
9,20240613,00117188,효성,Y,투자설명서,20240613000016,


In [91]:
import pandas as pd
import requests

API_KEY = '17db1c341b5874ad5ad8a43382af6b17d3009380'
URL = 'https://opendart.fss.or.kr/api/list.json'

params = {
    'crtfc_key': API_KEY,
    'pblntf_detail_ty': 'C004',  # 증권신고서(합병 등)
    'bgn_de': '20250501',
    'end_de': '20250610',
    'last_reprt_at': 'Y',
    'page_no': 1,
    'page_count': 100
}

resp = requests.get(URL, params=params)
resp.raise_for_status()
data = resp.json()

# 전체 목록을 데이터프레임으로 만들고 정렬
df = pd.DataFrame(data['list'])[['rcept_dt', 'corp_code', 'corp_name', 'corp_cls', 'report_nm', 'rcept_no', 'rm']]
df = df.sort_values(by='rcept_dt', ascending=True).reset_index(drop=True)

# "증권신고서(합병)" 문자열을 포함하는 행만 필터링
df_filtered = df[df['report_nm'].str.contains(r'증권신고서\(합병', na=False)]

df_filtered


,rcept_dt,corp_code,corp_name,corp_cls,report_nm,rcept_no,rm
2,20250519,00199252,HLB,K,[첨부정정]증권신고서(합병),20250519000332,
5,20250528,01693922,키움제8호스팩,K,[기재정정]증권신고서(합병),20250528000460,
6,20250604,01551042,애드포러스,E,[기재정정]증권신고서(합병),20250604000107,


In [11]:
import os
import requests

def download_dart_document(api_key: str, rcept_no: str) -> str:
    url = 'https://opendart.fss.or.kr/api/document.xml'
    params = {
        'crtfc_key': api_key,
        'rcept_no': rcept_no,
    }

    file_path = f'{rcept_no}.xml'  # 현재 디렉토리에 저장

    response = requests.get(url, params=params)
    response.raise_for_status()  # 요청 실패 시 예외 발생

    with open(file_path, 'wb') as f:
        f.write(response.content)

    print(f"📄 파일 저장 완료: {file_path}")
    return os.path.abspath(file_path)


# ✅ 예시 실행
if __name__ == "__main__":
    API_KEY = '17db1c341b5874ad5ad8a43382af6b17d3009380'
    RECEIPT_NO = '20250528000460'
    download_dart_document(API_KEY, RECEIPT_NO)

📄 파일 저장 완료: 20250528000460.xml


In [19]:
import requests

API_KEY = '17db1c341b5874ad5ad8a43382af6b17d3009380'
RECEIPT_NO = '20250528000460'
url = 'https://opendart.fss.or.kr/api/document.xml'
params = {'crtfc_key': API_KEY, 'rcept_no': RECEIPT_NO}

resp = requests.get(url, params=params)
resp.raise_for_status()

# 🔍 Content-Type 확인
print(f"📦 Content-Type: {resp.headers.get('Content-Type')}")

# 🔍 앞부분 바이트 확인
print(f"🧪 File signature (앞 8 bytes): {resp.content[:8]}")

# 🔍 파일 크기
print(f"📏 파일 크기: {len(resp.content)} bytes")

# 🔍 바이너리 저장해 보기
with open(f'{RECEIPT_NO}_raw.bin', 'wb') as f:
    f.write(resp.content)
print("✅ 바이너리 저장 완료")

📦 Content-Type: application/x-msdownload;charset=UTF-8
🧪 File signature (앞 8 bytes): b'PK\x03\x04\x14\x00\x08\x00'
📏 파일 크기: 566171 bytes
✅ 바이너리 저장 완료


In [22]:
import zipfile

RECEIPT_NO = '20250528000460'
zip_path = f'{RECEIPT_NO}_raw.bin'

with zipfile.ZipFile(zip_path, 'r') as zipf:
    # 내부 파일 목록 출력
    file_list = zipf.namelist()
    print("📂 ZIP 내부 파일 목록:")
    for name in file_list:
        print(f" - {name}")

    # 첫 번째 파일 미리보기 (앞 500자)
    target = file_list[0]
    print(f"\n📄 {target} 파일 내용 미리보기 (앞 500자):\n")
    with zipf.open(target) as f:
        preview = f.read(3000).decode('utf-8', errors='replace')
        print(preview)

📂 ZIP 내부 파일 목록:
 - 20250528000460.xml

📄 20250528000460.xml 파일 내용 미리보기 (앞 500자):

<?xml version="1.0" encoding="utf-8"?>


<DOCUMENT xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="dart4.xsd">

<DOCUMENT-NAME ACODE="10081">증권신고서(합병)</DOCUMENT-NAME>
<FORMULA-VERSION SUBVER="1" ADATE="20230220">6.0</FORMULA-VERSION>
<COMPANY-NAME AREGCIK="01693922">키움제8호기업인수목적 주식회사</COMPANY-NAME>


<BODY ATOCID="111">

<LIBRARY>
<CORRECTION>
<TITLE ATOC="Y" AASSOCNOTE="CORRECTION" ENG="Report of Disclosure Revision" ATOCID="110">정 정 신 고 (보고)</TITLE>

<TABLE ACLASS="NORMAL" AFIXTABLE="Y" WIDTH="600" BORDER="0">

<COLGROUP WIDTH="600">
<COL WIDTH="600"></COL>
</COLGROUP>

<TBODY>

<TR ACOPY="N" ADELETE="N" HEIGHT="30">
<TD WIDTH="591" HEIGHT="23"></TD>
</TR>

<TR ACOPY="N" ADELETE="N" HEIGHT="30">
<TD WIDTH="591" HEIGHT="23" ALIGN="RIGHT">2025년 05월 28일</TD>
</TR>
</TBODY>
</TABLE>
<P></P>
<P></P>

<TABLE ACLASS="NORMAL" AFIXTABLE="Y" WIDTH="600" BORDER="0">

<COLGROUP W

In [145]:
import zipfile
import io

RECEIPT_NO = '20250528000460'
zip_path = f'{RECEIPT_NO}_raw.bin'

API_KEY = '17db1c341b5874ad5ad8a43382af6b17d3009380'
RECEIPT_NO = '20250604000107'

# 1) 문서 다운로드 + 메모리 압축 해제
resp = requests.get(
    'https://opendart.fss.or.kr/api/document.xml',
    params={'crtfc_key': API_KEY, 'rcept_no': RECEIPT_NO}
)
resp.raise_for_status()
with zipfile.ZipFile(io.BytesIO(resp.content)) as z:
    with z.open(z.namelist()[0]) as f:
        xml = f.read().decode('utf-8', errors='replace')

# ✅ 특정 섹션 기준으로 내용 추출
section_title = "나. 회사의 현황"
start_idx = xml.find(section_title)

if start_idx == -1:
    print("❌ 해당 제목을 찾을 수 없습니다.")
else:
    # 이후 텍스트 일부 미리 출력
    preview = xml[start_idx:start_idx + 30000]
    print(f"\n📄 '{section_title}' 이후 내용 미리보기:\n")
    print(preview)


📄 '나. 회사의 현황' 이후 내용 미리보기:

나. 회사의 현황</P>

<P USERMARK="F-11 ">
<SPAN USERMARK="F-BT12 ">당사는 온라인 광고산업에서 광고주와 광고매체를 연결하는 모바일 광고 플랫폼을 주요 사업영역으로 하고 있습니다. </SPAN>
<SPAN USERMARK="F-BT12  0X0D0D0D">당사가 </SPAN>
<SPAN USERMARK="F-BT12  0X000000">보유한 모바일 광고 플랫폼은 링크마인, 바리스타, 그린피, 큐브마인, 엠플레이트 총 5가지입니다. </SPAN>
</P>
<P></P>
<P>링크마인과 바리스타는 SSP로서 매체사의 광고수익을 극대화하는 플랫폼으로 링크마인은 국내시장을, 바리스타는 해외시장을 타겟으로 서비스하고 있습니다. </P>
<P></P>

<P>
<SPAN USERMARK="0X0D0D0D">그린피는 당사가 보유한 AD Network로서 다양한 외부 AD Network, SSP, DSP와 연결되어 매체와 광고를 매칭하고 있습니다. 그린피는 특히 매체를 이용하는 사용자에게 </SPAN>
<SPAN USERMARK="0X000000">특정한 행동보상을 제공하여 광고주가 원하는 목표 성과를 달성하는 것에 특화된 플랫폼으로서 보상형 광고시장의 성장에 힘입어 2021년부터 2023년까지 2년간 매출액이 약 3배 성장하였습니다. 그린피의 경우 사용자에 대한 보상이 가능한 매체와 이러한 수요가 존재하는 광고주의 풀을 폭넓게 가져갈 필요가 있기 때문에 보상형 광고에 특화된 AD Network라는 플랫폼 형태를 띄고 있습니다.</SPAN>
</P>
<P></P>
<P>또한 당사는 광고주측 영업을 강화하기 위하여 DSP인 큐브마인을 개발하였으며 이를 통해 CPC(클릭당 비용, 광고를 클릭한 사용자에게만 비용을 지불하는 광고 모델), CPM(천 회 노출당 비용, 광고 노출 수에 따라 비용이 책정되는 모델) 방식의 광고를 집행하고 있습니다. 2024년에 큐브마인 광고지면의 실시간 광고입찰

## 파싱하기

In [141]:
import re
import io
import zipfile
import requests
from bs4 import BeautifulSoup
import pandas as pd

API_KEY = '17db1c341b5874ad5ad8a43382af6b17d3009380'
RECEIPT_NO = '20250604000107'

# 1) 문서 다운로드 + 메모리 압축 해제
resp = requests.get(
    'https://opendart.fss.or.kr/api/document.xml',
    params={'crtfc_key': API_KEY, 'rcept_no': RECEIPT_NO}
)
resp.raise_for_status()
with zipfile.ZipFile(io.BytesIO(resp.content)) as z:
    with z.open(z.namelist()[0]) as f:
        xml = f.read().decode('utf-8', errors='replace')

# 2) "1. 사업의 개요" 위치 찾기
section_title = "회사의 현황"
matches = [m.start() for m in re.finditer(re.escape(section_title), xml)]

# 3) 각 블록별 첫 번째 non-empty <p> 문단 추출
first_paras = []
for start, end in zip(matches, matches[1:] + [len(xml)]):
    block = xml[start:end]
    soup = BeautifulSoup(block, 'html.parser')
    para = None
    for p in soup.find_all('p'):
        text = p.get_text(strip=True)
        if text:  # 빈 문자열이 아니면 바로 선택
            para = text
            break
    first_paras.append(para)

# 4) 레이블 매핑 (첫째→합병회사, 둘째→피합병회사)
labels = ['합병회사', '피합병회사']
data = { label: first_paras[i] if i < len(first_paras) else None
         for i, label in enumerate(labels) }

# 5) DataFrame 생성
df = pd.DataFrame([data])
# print(df)
df

,합병회사,피합병회사
0,당사는 온라인 광고산업에서 광고주와 광고매체를 연결하는 모바일 광고 플랫폼을 주요 ...,"한편, 공급자 측면에서는 자동차산업의 공급자 범주가 전기ㆍ전자ㆍSW 등 이종산업기업..."


In [128]:
import re
import io
import zipfile
import requests
from bs4 import BeautifulSoup
import pandas as pd

API_KEY    = '17db1c341b5874ad5ad8a43382af6b17d3009380'
RECEIPT_NO = '20250528000460'

# 1) 문서 다운로드 + 메모리 압축 해제
resp = requests.get(
    'https://opendart.fss.or.kr/api/document.xml',
    params={'crtfc_key': API_KEY, 'rcept_no': RECEIPT_NO}
)
resp.raise_for_status()
with zipfile.ZipFile(io.BytesIO(resp.content)) as z:
    with z.open(z.namelist()[0]) as f:
        xml = f.read().decode('utf-8', errors='replace')

# 2) "1. 사업의 개요" 위치 모두 찾기
starts = [m.start() for m in re.finditer(re.escape("1. 사업의 개요"), xml)]
if not starts:
    raise ValueError("1. 사업의 개요 섹션을 찾을 수 없습니다.")

# 3) 각 블록 잘라내기 (다음 등장 전까지)
blocks = []
for i, s in enumerate(starts):
    e = starts[i+1] if i+1 < len(starts) else len(xml)
    blocks.append(xml[s:e])

# 4) 각 블록에서 종료 마커 전까지 자르고 <p>만 수집
end_marker = re.compile(r"(가\. 합병의 개요|2\. 주요 제품 및 서비스)")
all_paras = []
for block in blocks:
    # 종료 마커 위치 찾기
    m = end_marker.search(block)
    if m:
        block = block[:m.start()]

    soup = BeautifulSoup(block, 'html.parser')
    paras = [p.get_text(strip=True)
             for p in soup.find_all('p')
             if p.get_text(strip=True)]
    all_paras.append(paras)

# 5) 컬럼 이름에 매핑
data = {
    '합병회사': all_paras[0] if len(all_paras) > 0 else [],
    '피합병회사': all_paras[1] if len(all_paras) > 1 else []
}

# 6) 결과 DataFrame
df = pd.DataFrame([data])
df.to_excel('test2.xlsx')


### 사업개요 완성본

In [161]:
import re
import io
import zipfile
import requests
from bs4 import BeautifulSoup
import pandas as pd

API_KEY    = '17db1c341b5874ad5ad8a43382af6b17d3009380'
RECEIPT_NO = '20250225003558'

# 1) 문서 다운로드 + 메모리 압축 해제
resp = requests.get(
    'https://opendart.fss.or.kr/api/document.xml',
    params={'crtfc_key': API_KEY, 'rcept_no': RECEIPT_NO}
)
resp.raise_for_status()
with zipfile.ZipFile(io.BytesIO(resp.content)) as z:
    with z.open(z.namelist()[0]) as f:
        xml = f.read().decode('utf-8', errors='replace')

# 2) "1. 사업의 개요" 위치 모두 찾기
starts = [m.start() for m in re.finditer(re.escape("1. 사업의 개요"), xml)]
if not starts:
    raise ValueError("1. 사업의 개요 섹션을 찾을 수 없습니다.")

# 3) 각 블록 분리
blocks = []
for i, s in enumerate(starts):
    e = starts[i+1] if i+1 < len(starts) else len(xml)
    blocks.append(xml[s:e])

# 4) 각 블록에서 종료 마커 전까지 <p> 수집 → 하나의 문자열로 합치기
end_marker = re.compile(r"(합병의 개요|2\. 주요 제품 및 서비스)")
combined_paras = []
for block in blocks:
    # 종료 지점 자르기
    m = end_marker.search(block)
    cut_block = block[:m.start()] if m else block

    soup = BeautifulSoup(cut_block, 'html.parser')
    paras = [p.get_text(strip=True)
             for p in soup.find_all('p')
             if p.get_text(strip=True)]
    # 리스트를 하나의 문자열로 결합
    combined_paras.append(" ".join(paras))

# 5) 컬럼 매핑 및 DataFrame 생성
data = {
    '합병회사_사업개요': combined_paras[0] if len(combined_paras) > 0 else "",
    '피합병회사_사업개요': combined_paras[1] if len(combined_paras) > 1 else ""
}
df = pd.DataFrame([data])

# df.to_excel('test2.xlsx')
df

,합병회사_사업개요,피합병회사_사업개요
0,"당사는 스낵류, 액상차, 음료 등 식품 및 음료 제조, 판매 사업을 영위하고 있습니...",1. 합병에 관한 사항 당사는「자본시장과 금융투자업에 관한 법률」에 따라 회사의 주...


In [164]:
import re
import io
import zipfile
import requests
from bs4 import BeautifulSoup
import pandas as pd

API_KEY    = '17db1c341b5874ad5ad8a43382af6b17d3009380'
RECEIPT_NO = '20250604000107'

# 1) 문서 다운로드 + 메모리 압축 해제
resp = requests.get(
    'https://opendart.fss.or.kr/api/document.xml',
    params={'crtfc_key': API_KEY, 'rcept_no': RECEIPT_NO}
)
resp.raise_for_status()
with zipfile.ZipFile(io.BytesIO(resp.content)) as z:
    with z.open(z.namelist()[0]) as f:
        xml = f.read().decode('utf-8', errors='replace')

# 2) "1. 사업의 개요" 위치 모두 찾기
starts = [m.start() for m in re.finditer(re.escape("1. 사업의 개요"), xml)]
if not starts:
    raise ValueError("1. 사업의 개요 섹션을 찾을 수 없습니다.")

# 3) 각 블록 분리
blocks = []
for i, s in enumerate(starts):
    e = starts[i+1] if i+1 < len(starts) else len(xml)
    blocks.append(xml[s:e])

# 4) 각 블록별로:
#    - '회사의 현황' 이 있으면 첫 <p>만
#    - 아니면 end_marker 전까지 모든 <p> 문단 합치기
end_marker = re.compile(r"(합병의 개요|2\. 주요 제품 및 서비스)")
combined_paras = []

for block in blocks:
    if "나. 회사의 현황" in block:
        # 첫 번째 비어있지 않은 <p> 문단만 뽑기
        soup = BeautifulSoup(block, 'html.parser')
        first_para = ""
        for p in soup.find_all('p'):
            text = p.get_text(strip=True)
            if text:
                first_para = text
                break
        combined_paras.append(first_para)

    else:
        # 기존처럼 end_marker 전까지 모든 문단 합치기
        m = end_marker.search(block)
        cut_block = block[:m.start()] if m else block

        soup = BeautifulSoup(cut_block, 'html.parser')
        paras = [
            p.get_text(strip=True)
            for p in soup.find_all('p')
            if p.get_text(strip=True)
        ]
        combined_paras.append(" ".join(paras))

# 5) 컬럼 매핑 및 DataFrame 생성
data = {
    '합병회사_사업개요': combined_paras[0] if len(combined_paras) > 0 else "",
    '피합병회사_사업개요': combined_paras[1] if len(combined_paras) > 1 else ""
}
df = pd.DataFrame([data])

# print(df)
df

,합병회사_사업개요,피합병회사_사업개요
0,가. 용어설명,가. 합병에 관한 사항당사는「자본시장과 금융투자업에 관한 법률」에 따라 회사의 주권...


In [165]:
import re
from bs4 import BeautifulSoup

# xml: 이미 읽어온 전체 XML 문자열
# 1) "1. 사업의 개요" 블록 시작
start_idx = xml.find("1. 사업의 개요")
if start_idx == -1:
    raise ValueError("1. 사업의 개요 섹션을 찾을 수 없습니다.")
block = xml[start_idx:]  # 종료 마커 없이 끝까지

# 2) '회사의 현황' 검사
if "회사의 현황" in block:
    # '회사의 현황' 직후부터 파싱
    sub = block.split("회사의 현황", 1)[1]
else:
    # '1. 사업의 개요' 직후부터 파싱
    sub = block.split("1. 사업의 개요", 1)[1]

# 3) BeautifulSoup 으로 첫 <p> 문단 추출
soup = BeautifulSoup(sub, "html.parser")
first_para = ""
for p in soup.find_all("p"):
    text = p.get_text(strip=True)
    if text:
        first_para = text
        break

print(first_para)

당사는 온라인 광고산업에서 광고주와 광고매체를 연결하는 모바일 광고 플랫폼을 주요 사업영역으로 하고 있습니다.당사가보유한 모바일 광고 플랫폼은 링크마인, 바리스타, 그린피, 큐브마인, 엠플레이트 총 5가지입니다.


In [168]:
import re
import io
import zipfile
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd

API_KEY = '17db1c341b5874ad5ad8a43382af6b17d3009380'

def clean_number(val: str) -> int | None:
    return int(re.sub(r'[^\d]', '', val)) if isinstance(val, str) and re.search(r'\d', val) else None

def parse_merger_overview(rcept_no: str) -> dict | None:
    # 1) Download & unzip
    url = 'https://opendart.fss.or.kr/api/document.xml'
    resp = requests.get(url, params={'crtfc_key': API_KEY, 'rcept_no': rcept_no})
    if resp.status_code != 200:
        return None
    try:
        with zipfile.ZipFile(io.BytesIO(resp.content)) as z:
            with z.open(z.namelist()[0]) as f:
                xml = f.read().decode('utf-8', errors='replace')

        # 2) Merge overview parsing
        part = xml.split("(1) 합병 당사회사의 개요", 1)
        if len(part) < 2:
            return None
        block = "(1) 합병 당사회사의 개요" + part[1]
        soup = BeautifulSoup(block, 'html.parser')
        table = soup.find('table')
        rows = table.find_all('tr') if table else []

        merger_data = {}
        keys = {
            '법인명': 'corp_name',
            '납입자본금(주1)': 'capital',
            '자산총액(주2)': 'assets',
            '주권상장여부': 'listing',
            '발행주식의 종류 및 수': 'shares'
        }
        for tr in rows:
            cols = tr.find_all('td')
            texts = [c.get_text(strip=True) for c in cols]
            if len(texts) == 3 and texts[0] in keys:
                merger_data[keys[texts[0]]] = {
                    '합병법인': texts[1],
                    '피합병법인': texts[2]
                }

        # 3) Business overview parsing
        starts = [m.start() for m in re.finditer(re.escape("1. 사업의 개요"), xml)]
        blocks = []
        for i, s in enumerate(starts):
            e = starts[i+1] if i+1 < len(starts) else len(xml)
            blocks.append(xml[s:e])

        end_marker = re.compile(
           r"(?:1\. 합병의 개요|가\. 합병의 개요|나\. 합병의 개요|2\. 주요 제품 및 서비스)"
        )
        combined_paras = []

        for blk in blocks:
            if "나. 회사의 현황" in blk:
                # after '회사의 현황', first <p>
                sub = blk.split("나. 회사의 현황", 1)[1]
                soup2 = BeautifulSoup(sub, 'html.parser')
                para = ""
                for p in soup2.find_all('p'):
                    txt = p.get_text(strip=True)
                    if txt:
                        para = txt
                        break
                combined_paras.append(para)
            else:
                # until end_marker, then merge all <p>
                m = end_marker.search(blk)
                cut = blk[:m.start()] if m else blk
                soup2 = BeautifulSoup(cut, 'html.parser')
                paras = [p.get_text(strip=True) for p in soup2.find_all('p') if p.get_text(strip=True)]
                combined_paras.append(" ".join(paras))

        business_merge = combined_paras[0] if len(combined_paras) > 0 else ""
        business_target = combined_paras[1] if len(combined_paras) > 1 else ""

        # 4) Build result
        def G(d, f, r): return d.get(f, {}).get(r)
        def N(d, f, r): return clean_number(G(d, f, r))

        return {
            '공시회사':             G(merger_data, 'corp_name', '합병법인'),
            '피합병법인':           G(merger_data, 'corp_name', '피합병법인'),
            '납입자본금(합병)':     N(merger_data, 'capital', '합병법인'),
            '납입자본금(피합병)':   N(merger_data, 'capital', '피합병법인'),
            '자산총액(합병)':       N(merger_data, 'assets', '합병법인'),
            '자산총액(피합병)':     N(merger_data, 'assets', '피합병법인'),
            '합병법인 상장':        G(merger_data, 'listing', '합병법인'),
            '피합병법인 상장':      G(merger_data, 'listing', '피합병법인'),
            '발행주식수(합병)':     N(merger_data, 'shares', '합병법인'),
            '발행주식수(피합병)':   N(merger_data, 'shares', '피합병법인'),
            '사업개요(합병)':    business_merge,
            '사업개요(피합병)':  business_target
        }
    except Exception:
        return None

def get_merger_reports():
    url = 'https://opendart.fss.or.kr/api/list.json'
    params = {
        'crtfc_key': API_KEY,
        'pblntf_detail_ty': 'C004',
        'bgn_de': '20250501',
        'end_de': '20250610',
        'last_reprt_at': 'Y',
        'page_no': 1,
        'page_count': 100
    }
    resp = requests.get(url, params=params)
    data = resp.json()
    df = pd.DataFrame(data['list'])[['rcept_dt', 'rcept_no', 'corp_name', 'report_nm']]
    df = df[df['report_nm'].str.contains(r'증권신고서\(합병', na=False)].reset_index(drop=True)

    results = []
    for _, row in tqdm(df.iterrows(), total=len(df), desc="📄 파싱 중"):
        parsed = parse_merger_overview(row['rcept_no'])
        if parsed:
            parsed['날짜'] = row['rcept_dt']
            results.append(parsed)

    return pd.DataFrame(results)

# Execute
df_all = get_merger_reports()
df_all


📄 파싱 중: 100%|██████████| 3/3 [00:03<00:00,  1.12s/it]


,공시회사,피합병법인,납입자본금(합병),납입자본금(피합병),자산총액(합병),자산총액(피합병),합병법인 상장,피합병법인 상장,발행주식수(합병),발행주식수(피합병),사업개요(합병),사업개요(피합병),날짜
0,주식회사 애드포러스,신영해피투모로우제8호기업인수목적주식회사,2140680000,676000000,24881222524,15608171880,None,None,4281360,6760000,당사는 온라인 광고산업에서 광고주와 광고매체를 연결하는 모바일 광고 플랫폼을 주요 ...,가. 합병에 관한 사항당사는「자본시장과 금융투자업에 관한 법률」에 따라 회사의 주권...,20250604
1,키움제8호기업인수목적 주식회사,주식회사 지슨,588000000,8664789500,12311598310,15106836691,상장(코스닥),상장(코넥스),5880000,17329579,당사는 자본시장과 금융투자업에 관한 법률 시행령(이하 법 시행령) 제6조 제4항 제...,당사는 2000년에 설립된 국내 최고의 전파분석 기술을 보유한 보안솔루션 기업으로서...,20250528


In [169]:
df_all.to_excel('test3.xlsx')

In [112]:
import pandas as pd
import io
import zipfile
import requests
from bs4 import BeautifulSoup
from openpyxl.styles import Alignment

API_KEY    = '17db1c341b5874ad5ad8a43382af6b17d3009380'
RECEIPT_NO = '20250225003558'

# 1) 문서 다운로드 & 메모리 압축 해제
resp = requests.get(
    'https://opendart.fss.or.kr/api/document.xml',
    params={'crtfc_key': API_KEY, 'rcept_no': RECEIPT_NO}
)
resp.raise_for_status()
with zipfile.ZipFile(io.BytesIO(resp.content)) as z:
    with z.open(z.namelist()[0]) as f:
        xml = f.read().decode('utf-8', errors='replace')

# 2) '1. 사업의 개요' ~ '2. 주요 제품 및 서비스' 사이 블록 추출
start = xml.find("1. 사업의 개요")
end   = xml.find("2. 주요 제품 및 서비스")
if start == -1:
    raise ValueError("1. 사업의 개요 섹션을 찾을 수 없습니다.")
if end == -1:
    end = len(xml)
block = xml[start:end]

# 3) 모든 <p> 문단 추출 (빈 문단 제외)
soup  = BeautifulSoup(block, 'html.parser')
paras = [p.get_text(strip=True) for p in soup.find_all('p') if p.get_text(strip=True)]

# 4) 줄바꿈('\n')으로 합치기
combined = "\n".join(paras)

# 5) DataFrame에 한 행으로 저장
df = pd.DataFrame({'사업 개요': [combined]})

# 6) to_excel + openpyxl 엔진 사용, wrapText 적용
with pd.ExcelWriter('test1_0624.xlsx', engine='openpyxl') as writer:
    df.to_excel(writer, index=False, sheet_name='Sheet1')
    workbook  = writer.book
    worksheet = writer.sheets['Sheet1']

    # A열 너비 조정
    # worksheet.column_dimensions['A'].width = 50

    # A열의 모든 셀에 wrapText 적용
    for cell in worksheet['A']:
        cell.alignment = Alignment(wrap_text=True)

# print("✅ test1_0624.xlsx 생성 완료 (openpyxl 엔진, wrapText 적용)")

In [110]:
df

,사업 개요
0,"당사는 스낵류, 액상차, 음료 등 식품 및 음료 제조, 판매 사업을 영위하고 있습니..."


In [31]:
from bs4 import BeautifulSoup
import pandas as pd

# block: "(1) 합병 당사회사의 개요" 포함된 블록 문자열
soup = BeautifulSoup(block, 'html.parser')

# 모든 테이블 중 첫 번째 테이블만 사용
table = soup.find('table')
rows = table.find_all('tr')

result = {}
target_keys = [
    '법인명', '합병 후 존속여부', '납입자본금(주1)', 
    '자산총액(주2)', '주권상장여부', '발행주식의 종류 및 수'
]

for row in rows:
    cols = row.find_all('td')
    texts = [col.get_text(strip=True) for col in cols]
    if len(texts) == 3 and texts[0] in target_keys:
        result[texts[0]] = {
            '합병법인': texts[1],
            '피합병법인': texts[2]
        }

# DataFrame으로 변환
df = pd.DataFrame(result).T.reset_index().rename(columns={'index': '구분'})
print(df)


             구분              합병법인            피합병법인
0           법인명  키움제8호기업인수목적 주식회사          주식회사 지슨
1     합병 후 존속여부                존속               소멸
2     납입자본금(주1)      588,000,000원   8,664,789,500원
3      자산총액(주2)   12,311,598,310원  15,106,836,691원
4        주권상장여부           상장(코스닥)          상장(코넥스)
5  발행주식의 종류 및 수    보통주 5,880,000주  보통주 17,329,579주


In [32]:
import zipfile
import re
from bs4 import BeautifulSoup
import pandas as pd

RECEIPT_NO = '20250528000460'
zip_path = f'{RECEIPT_NO}_raw.bin'

# 1. ZIP 압축 열기 및 XML 추출
with zipfile.ZipFile(zip_path, 'r') as zipf:
    target = zipf.namelist()[0]
    with zipf.open(target) as f:
        xml_content = f.read().decode('utf-8', errors='replace')

# 2. "(1) 합병 당사회사의 개요" 블록 추출
section_title = "(1) 합병 당사회사의 개요"
start = xml_content.find(section_title)
end = len(xml_content)
block = xml_content[start:end]

# 3. 필요한 정보 추출
soup = BeautifulSoup(block, 'html.parser')
table = soup.find('table')  # 첫 번째 테이블

rows = table.find_all('tr')
result = {}
target_keys = [
    '법인명', '합병 후 존속여부', '납입자본금(주1)', 
    '자산총액(주2)', '주권상장여부', '발행주식의 종류 및 수'
]

for row in rows:
    cols = row.find_all('td')
    texts = [col.get_text(strip=True) for col in cols]
    if len(texts) == 3 and texts[0] in target_keys:
        result[texts[0]] = {
            '합병법인': texts[1],
            '피합병법인': texts[2]
        }

# 4. DataFrame 생성
df = pd.DataFrame(result).T.reset_index().rename(columns={'index': '구분'})
# print(df)
df

,구분,합병법인,피합병법인
0,법인명,키움제8호기업인수목적 주식회사,주식회사 지슨
1,합병 후 존속여부,존속,소멸
2,납입자본금(주1),"588,000,000원","8,664,789,500원"
3,자산총액(주2),"12,311,598,310원","15,106,836,691원"
4,주권상장여부,상장(코스닥),상장(코넥스)
5,발행주식의 종류 및 수,"보통주 5,880,000주","보통주 17,329,579주"


In [38]:
import zipfile
import re
from bs4 import BeautifulSoup
import pandas as pd

RECEIPT_NO = '20250528000460'
zip_path = f'{RECEIPT_NO}_raw.bin'

# 1. ZIP에서 XML 꺼내기
with zipfile.ZipFile(zip_path, 'r') as zipf:
    target = zipf.namelist()[0]
    with zipf.open(target) as f:
        xml_content = f.read().decode('utf-8', errors='replace')

# 2. "가. 회사의 개황" 블록 추출
section_title = "4. 합병의 상대방 회사"
start = xml_content.find(section_title)
end = xml_content.find("나. 주요 재무에 관한 사항")  # 다음 항목이 있을 경우
if end == -1:
    end = len(xml_content)
block = xml_content[start:end]

# 3. BeautifulSoup 파싱
soup = BeautifulSoup(block, 'html.parser')
text = soup.get_text(separator="\n", strip=True)

# 4. 업종 및 주요사업 내용 추출 시도
lines = text.splitlines()

industry = ""
business = ""
for i, line in enumerate(lines):
    if "업종" in line:
        industry = lines[i+1] if i+1 < len(lines) else ""
    if "주요사업의 내용" in line:
        business = lines[i+1] if i+1 < len(lines) else ""

# 5. 출력
print(f"🧾 업종: {industry}")
print(f"🧾 주요사업의 내용: {business}")


🧾 업종: 해당사항 없습니다.
🧾 주요사업의 내용: 당사는 무선보안솔루션 단일사업부문으로 구성되어 있으며, 상시형무선도청탐지시스템, 상시형무선해킹탐지시스템 및 상시형무선불법촬영탐지시스템의 개발, 제조 및판매를 주요사업목적으로 하고 있습니다.기타의 자세한 사항은 동 보고서의 "Ⅱ사업의 내용"  을 참조하시기 바랍니다.


In [41]:
from bs4 import BeautifulSoup
import zipfile

RECEIPT_NO = '20250528000460'
zip_path = f'{RECEIPT_NO}_raw.bin'

# 1. ZIP 압축 해제 및 XML 읽기
with zipfile.ZipFile(zip_path, 'r') as zipf:
    target = zipf.namelist()[0]
    with zipf.open(target) as f:
        xml_content = f.read().decode('utf-8', errors='replace')

# 2. "가. 회사의 개황" 블록 추출
section_title = "가. 회사의 개황"
start = xml_content.find(section_title)
end = xml_content.find("나. 주요 재무에 관한 사항")
if end == -1:
    end = len(xml_content)
block = xml_content[start:end]

# 3. BeautifulSoup 파싱
soup = BeautifulSoup(block, 'html.parser')
table = soup.find('table')  # 첫 번째 테이블

# 4. 원하는 항목 추출
industry = ""
business = ""

for row in table.find_all('tr'):
    cells = row.find_all('td')
    if not cells or len(cells) < 2:
        continue
    label = cells[0].get_text(strip=True)
    value = cells[1].get_text(strip=True)

    if "업종" in label:
        industry = value
    elif "주요사업" in label:
        business = value

# 5. 결과 출력
print("📌 [가. 회사의 개황] 파싱 결과")
print(f"🧾 업종: {industry}")
print(f"🧾 주요사업의 내용: {business}")

📌 [가. 회사의 개황] 파싱 결과
🧾 업종: 기타 측정, 시험, 항해, 제어 및 정밀기기 제조업
🧾 주요사업의 내용: 상시형 무선도청 탐지시스템, 상시형 무선백도어 탐지시스템, 상시형 불법촬영 탐지시스템 등 무선보안솔루션


In [42]:
import zipfile
from bs4 import BeautifulSoup
import pandas as pd

RECEIPT_NO = '20250528000460'
zip_path = f'{RECEIPT_NO}_raw.bin'

# 1. ZIP 압축 열기 및 XML 추출
with zipfile.ZipFile(zip_path, 'r') as zipf:
    target = zipf.namelist()[0]
    with zipf.open(target) as f:
        xml_content = f.read().decode('utf-8', errors='replace')

# -------------------------
# 📌 (1) 합병 당사회사의 개요
# -------------------------
block_merger = xml_content[
    xml_content.find("(1) 합병 당사회사의 개요") :
]

soup_merger = BeautifulSoup(block_merger, 'html.parser')
table_merger = soup_merger.find('table')

rows = table_merger.find_all('tr')
result = {}
target_keys = [
    '법인명', '합병 후 존속여부', '납입자본금(주1)',
    '자산총액(주2)', '주권상장여부', '발행주식의 종류 및 수'
]

for row in rows:
    cols = row.find_all('td')
    texts = [col.get_text(strip=True) for col in cols]
    if len(texts) == 3 and texts[0] in target_keys:
        result[texts[0]] = {
            '합병법인': texts[1],
            '피합병법인': texts[2]
        }

df_merger = pd.DataFrame(result).T.reset_index().rename(columns={'index': '구분'})

# -------------------------
# 📌 가. 회사의 개황
# -------------------------
start = xml_content.find("가. 회사의 개황")
end = xml_content.find("나. 주요 재무에 관한 사항")
if end == -1:
    end = len(xml_content)
block_overview = xml_content[start:end]

soup_overview = BeautifulSoup(block_overview, 'html.parser')
table_overview = soup_overview.find('table')

# 추출 대상
industry = ""
business = ""

for row in table_overview.find_all('tr'):
    cells = row.find_all('td')
    if not cells or len(cells) < 2:
        continue
    label = cells[0].get_text(strip=True)
    value = cells[1].get_text(strip=True)

    if "업종" in label:
        industry = value
    elif "주요사업" in label:
        business = value

df_overview = pd.DataFrame({
    '구분': ['업종', '주요사업의 내용'],
    '내용': [industry, business]
})

# -------------------------
# ✅ 결과 출력 (선택)
# -------------------------
print("📋 합병 당사회사의 개요:")
print(df_merger, "\n")

print("📋 회사의 개황:")
print(df_overview)


📋 합병 당사회사의 개요:
             구분              합병법인            피합병법인
0           법인명  키움제8호기업인수목적 주식회사          주식회사 지슨
1     합병 후 존속여부                존속               소멸
2     납입자본금(주1)      588,000,000원   8,664,789,500원
3      자산총액(주2)   12,311,598,310원  15,106,836,691원
4        주권상장여부           상장(코스닥)          상장(코넥스)
5  발행주식의 종류 및 수    보통주 5,880,000주  보통주 17,329,579주 

📋 회사의 개황:
         구분                                                 내용
0        업종                       기타 측정, 시험, 항해, 제어 및 정밀기기 제조업
1  주요사업의 내용  상시형 무선도청 탐지시스템, 상시형 무선백도어 탐지시스템, 상시형 불법촬영 탐지시스...


## 완성 (파라미터: 공시문서 번호)

In [148]:
import re
import zipfile
import io
import requests
from bs4 import BeautifulSoup
import pandas as pd

API_KEY = '17db1c341b5874ad5ad8a43382af6b17d3009380'
RECEIPT_NO = '20250604000107'

# 1. 숫자 문자열 정리 함수
def clean_number(val: str) -> int | None:
    return int(re.sub(r'[^\d]', '', val)) if isinstance(val, str) and re.search(r'\d', val) else None

# 2. OpenDART에서 ZIP(XML) 파일 메모리로 다운로드
url = 'https://opendart.fss.or.kr/api/document.xml'
params = {'crtfc_key': API_KEY, 'rcept_no': RECEIPT_NO}
response = requests.get(url, params=params)
response.raise_for_status()

# 3. 압축 해제 후 XML 추출 (메모리 기반 처리)
with zipfile.ZipFile(io.BytesIO(response.content)) as zipf:
    with zipf.open(zipf.namelist()[0]) as f:
        xml_content = f.read().decode('utf-8', errors='replace')

# 4. (1) 합병 당사회사의 개요 파싱
block_merger = xml_content[xml_content.find("(1) 합병 당사회사의 개요"):]
soup_merger = BeautifulSoup(block_merger, 'html.parser')
table_merger = soup_merger.find('table')

rows = table_merger.find_all('tr') if table_merger else []
merger_data = {}
target_keys = {
    '법인명': 'corp_name',
    '납입자본금(주1)': 'capital',
    '자산총액(주2)': 'assets',
    '주권상장여부': 'listing',
    '발행주식의 종류 및 수': 'shares'
}

for row in rows:
    cols = row.find_all('td')
    texts = [col.get_text(strip=True) for col in cols]
    if len(texts) == 3 and texts[0] in target_keys:
        merger_data[target_keys[texts[0]]] = {
            '합병법인': texts[1],
            '피합병법인': texts[2]
        }

# 5. 가. 회사의 개황 → 업종, 주요사업 파싱
start = xml_content.find("가. 회사의 개황")
end = xml_content.find("나. 주요 재무에 관한 사항")
block_overview = xml_content[start:end if end != -1 else len(xml_content)]
soup_overview = BeautifulSoup(block_overview, 'html.parser')
table_overview = soup_overview.find('table')

industry, business = None, None
if table_overview:
    for row in table_overview.find_all('tr'):
        cells = row.find_all('td')
        if not cells or len(cells) < 2:
            continue
        label = cells[0].get_text(strip=True)
        value = cells[1].get_text(strip=True)
        if "업종" in label:
            industry = value
        elif "주요사업" in label:
            business = value

# 6. 병합된 row 구성 함수
def get_field(data, field, role):
    return clean_number(data[field][role]) if field in data and role in data[field] else None

def get_raw(data, field, role):
    return data[field][role] if field in data and role in data[field] else None

final_row = {
    '공시회사': get_raw(merger_data, 'corp_name', '합병법인'),
    '피합병법인': get_raw(merger_data, 'corp_name', '피합병법인'),
    '납입자본금(피합병)': get_field(merger_data, 'capital', '피합병법인'),
    '납입자본금(합병)': get_field(merger_data, 'capital', '합병법인'),
    '자산총액(피합병)': get_field(merger_data, 'assets', '피합병법인'),
    '자산총액(합병)': get_field(merger_data, 'assets', '합병법인'),
    '피합병법인 상장': get_raw(merger_data, 'listing', '피합병법인'),
    '합병법인 상장': get_raw(merger_data, 'listing', '합병법인'),
    '발행주식수(피합병)': get_field(merger_data, 'shares', '피합병법인'),
    '발행주식수(합병)': get_field(merger_data, 'shares', '합병법인'),
    '업종': industry,
    '주요사업': business
}

# 7. 결과 출력
df_summary = pd.DataFrame([final_row])

In [149]:
df_summary

,공시회사,피합병법인,납입자본금(피합병),납입자본금(합병),자산총액(피합병),자산총액(합병),피합병법인 상장,합병법인 상장,발행주식수(피합병),발행주식수(합병),업종,주요사업
0,주식회사 애드포러스,신영해피투모로우제8호기업인수목적주식회사,676000000,2140680000,15608171880,24881222524,None,None,6760000,4281360,그외 기타 금융 지원 서비스업,한국거래소의 코스닥시장에 상장한 후 다른 회사와 합병


## 정리

In [150]:
import re
import io
import zipfile
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup

API_KEY = '17db1c341b5874ad5ad8a43382af6b17d3009380'

def clean_number(val: str) -> int | None:
    return int(re.sub(r'[^\d]', '', val)) if isinstance(val, str) and re.search(r'\d', val) else None

def parse_merger_overview(rcept_no: str) -> dict | None:
    # 1) 문서 다운로드
    url = 'https://opendart.fss.or.kr/api/document.xml'
    resp = requests.get(url, params={'crtfc_key': API_KEY, 'rcept_no': rcept_no})
    if resp.status_code != 200:
        return None

    try:
        # 2) 메모리에서 ZIP 압축 해제
        with zipfile.ZipFile(io.BytesIO(resp.content)) as z:
            # 내부 XML 파일 읽기
            with z.open(z.namelist()[0]) as f:
                xml_content = f.read().decode('utf-8', errors='replace')

        # 3) (1) 합병 당사회사의 개요 파싱
        block_merger = xml_content.split("(1) 합병 당사회사의 개요", 1)
        if len(block_merger) < 2:
            return None
        block_merger = "(1) 합병 당사회사의 개요" + block_merger[1]

        soup_merger = BeautifulSoup(block_merger, 'html.parser')
        table_merger = soup_merger.find('table')
        rows = table_merger.find_all('tr') if table_merger else []

        merger_data = {}
        keys = {
            '법인명': 'corp_name',
            '납입자본금(주1)': 'capital',
            '자산총액(주2)': 'assets',
            '주권상장여부': 'listing',
            '발행주식의 종류 및 수': 'shares'
        }
        for tr in rows:
            td = tr.find_all('td')
            texts = [c.get_text(strip=True) for c in td]
            if len(texts) == 3 and texts[0] in keys:
                merger_data[keys[texts[0]]] = {
                    '합병법인': texts[1],
                    '피합병법인': texts[2]
                }

        # 4) 가. 회사의 개황 파싱
        part = xml_content.split("가. 회사의 개황", 1)
        if len(part) < 2:
            industry = business = None
        else:
            block_overview = "가. 회사의 개황" + part[1]
            block_overview = block_overview.split("나. 주요 재무에 관한 사항", 1)[0]
            soup_overview = BeautifulSoup(block_overview, 'html.parser')
            table_overview = soup_overview.find('table')
            industry = business = None
            if table_overview:
                for tr in table_overview.find_all('tr'):
                    td = tr.find_all('td')
                    if len(td) >= 2:
                        label, val = td[0].get_text(strip=True), td[1].get_text(strip=True)
                        if "업종" in label:
                            industry = val
                        elif "주요사업" in label:
                            business = val

        # 5) 결과 반환
        def get(d, f, r): return d.get(f, {}).get(r)
        def getn(d, f, r): return clean_number(get(d, f, r))

        return {
            '공시회사': get(merger_data, 'corp_name', '합병법인'),
            '피합병법인': get(merger_data, 'corp_name', '피합병법인'),
            '납입자본금(합병)': getn(merger_data, 'capital', '합병법인'),
            '납입자본금(피합병)': getn(merger_data, 'capital', '피합병법인'),
            '자산총액(합병)': getn(merger_data, 'assets', '합병법인'),
            '자산총액(피합병)': getn(merger_data, 'assets', '피합병법인'),
            '합병법인 상장': get(merger_data, 'listing', '합병법인'),
            '피합병법인 상장': get(merger_data, 'listing', '피합병법인'),
            '발행주식수(합병)': getn(merger_data, 'shares', '합병법인'),
            '발행주식수(피합병)': getn(merger_data, 'shares', '피합병법인'),
            '업종': industry,
            '주요사업': business
        }

    except Exception:
        return None

# 전체 리스트 수집 및 파싱
def get_merger_reports():
    url = 'https://opendart.fss.or.kr/api/list.json'
    params = {
        'crtfc_key': API_KEY,
        'pblntf_detail_ty': 'C004',  # 증권신고서(합병)
        'bgn_de': '20250501',
        'end_de': '20250610',
        'last_reprt_at': 'Y',
        'page_no': 1,
        'page_count': 100
    }

    resp = requests.get(url, params=params)
    data = resp.json()
    df = pd.DataFrame(data['list'])[['rcept_dt', 'rcept_no', 'corp_name', 'report_nm']]
    df = df[df['report_nm'].str.contains(r'증권신고서\(합병', na=False)].reset_index(drop=True)

    results = []
    for _, row in tqdm(df.iterrows(), total=len(df), desc="📄 파싱 중"):
        parsed = parse_merger_overview(row['rcept_no'])
        if parsed:
            parsed['rcept_dt'] = row['rcept_dt']
            results.append(parsed)

    df = pd.DataFrame(results)
    df = df.rename(columns={'rcept_dt': '날짜'})
    return df

In [151]:
df_all = get_merger_reports()
df_all

📄 파싱 중: 100%|██████████| 3/3 [00:05<00:00,  1.71s/it]


,공시회사,피합병법인,납입자본금(합병),납입자본금(피합병),자산총액(합병),자산총액(피합병),합병법인 상장,피합병법인 상장,발행주식수(합병),발행주식수(피합병),업종,주요사업,날짜
0,주식회사 애드포러스,신영해피투모로우제8호기업인수목적주식회사,2140680000,676000000,24881222524,15608171880,None,None,4281360,6760000,그외 기타 금융 지원 서비스업,한국거래소의 코스닥시장에 상장한 후 다른 회사와 합병,20250604
1,키움제8호기업인수목적 주식회사,주식회사 지슨,588000000,8664789500,12311598310,15106836691,상장(코스닥),상장(코넥스),5880000,17329579,"기타 측정, 시험, 항해, 제어 및 정밀기기 제조업","상시형 무선도청 탐지시스템, 상시형 무선백도어 탐지시스템, 상시형 불법촬영 탐지시스...",20250528


In [152]:
import re
import io
import zipfile
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd

API_KEY = '17db1c341b5874ad5ad8a43382af6b17d3009380'

def clean_number(val: str) -> int | None:
    return int(re.sub(r'[^\d]', '', val)) if isinstance(val, str) and re.search(r'\d', val) else None

def parse_merger_overview(rcept_no: str) -> dict | None:
    # 1) 문서 다운로드
    url = 'https://opendart.fss.or.kr/api/document.xml'
    resp = requests.get(url, params={'crtfc_key': API_KEY, 'rcept_no': rcept_no})
    if resp.status_code != 200:
        return None

    try:
        # 2) 메모리에서 ZIP 압축 해제
        with zipfile.ZipFile(io.BytesIO(resp.content)) as z:
            with z.open(z.namelist()[0]) as f:
                xml_content = f.read().decode('utf-8', errors='replace')

        # 3) (1) 합병 당사회사의 개요 파싱
        part = xml_content.split("(1) 합병 당사회사의 개요", 1)
        if len(part) < 2:
            return None
        block = "(1) 합병 당사회사의 개요" + part[1]
        soup = BeautifulSoup(block, 'html.parser')
        table = soup.find('table')
        rows = table.find_all('tr') if table else []

        merger_data = {}
        keys = {
            '법인명': 'corp_name',
            '납입자본금(주1)': 'capital',
            '자산총액(주2)': 'assets',
            '주권상장여부': 'listing',
            '발행주식의 종류 및 수': 'shares'
        }
        for tr in rows:
            cols = tr.find_all('td')
            texts = [c.get_text(strip=True) for c in cols]
            if len(texts) == 3 and texts[0] in keys:
                merger_data[keys[texts[0]]] = {
                    '합병법인': texts[1],
                    '피합병법인': texts[2]
                }

        # 4) 결과 dict — 이제 업종·주요사업은 빠집니다
        def G(d, f, r): return d.get(f, {}).get(r)
        def N(d, f, r): return clean_number(G(d, f, r))

        return {
            '공시회사': G(merger_data, 'corp_name', '합병법인'),
            '피합병법인': G(merger_data, 'corp_name', '피합병법인'),
            '납입자본금(합병)': N(merger_data, 'capital', '합병법인'),
            '납입자본금(피합병)': N(merger_data, 'capital', '피합병법인'),
            '자산총액(합병)': N(merger_data, 'assets', '합병법인'),
            '자산총액(피합병)': N(merger_data, 'assets', '피합병법인'),
            '합병법인 상장': G(merger_data, 'listing', '합병법인'),
            '피합병법인 상장': G(merger_data, 'listing', '피합병법인'),
            '발행주식수(합병)': N(merger_data, 'shares', '합병법인'),
            '발행주식수(피합병)': N(merger_data, 'shares', '피합병법인'),
        }

    except Exception:
        return None

def get_merger_reports():
    url = 'https://opendart.fss.or.kr/api/list.json'
    params = {
        'crtfc_key': API_KEY,
        'pblntf_detail_ty': 'C004',
        'bgn_de': '20250501',
        'end_de': '20250610',
        'last_reprt_at': 'Y',
        'page_no': 1,
        'page_count': 100
    }

    resp = requests.get(url, params=params)
    data = resp.json()
    df = pd.DataFrame(data['list'])[['rcept_dt', 'rcept_no', 'corp_name', 'report_nm']]
    df = df[df['report_nm'].str.contains(r'증권신고서\(합병', na=False)].reset_index(drop=True)

    results = []
    for _, row in tqdm(df.iterrows(), total=len(df), desc="📄 파싱 중"):
        parsed = parse_merger_overview(row['rcept_no'])
        if parsed:
            parsed['날짜'] = row['rcept_dt']
            results.append(parsed)

    return pd.DataFrame(results)

# 실행
df_all = get_merger_reports()

📄 파싱 중: 100%|██████████| 3/3 [00:02<00:00,  1.03it/s]


In [153]:
df_all

,공시회사,피합병법인,납입자본금(합병),납입자본금(피합병),자산총액(합병),자산총액(피합병),합병법인 상장,피합병법인 상장,발행주식수(합병),발행주식수(피합병),날짜
0,주식회사 애드포러스,신영해피투모로우제8호기업인수목적주식회사,2140680000,676000000,24881222524,15608171880,None,None,4281360,6760000,20250604
1,키움제8호기업인수목적 주식회사,주식회사 지슨,588000000,8664789500,12311598310,15106836691,상장(코스닥),상장(코넥스),5880000,17329579,20250528


In [156]:
import re
import io
import zipfile
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd

API_KEY = '17db1c341b5874ad5ad8a43382af6b17d3009380'

def clean_number(val: str) -> int | None:
    return int(re.sub(r'[^\d]', '', val)) if isinstance(val, str) and re.search(r'\d', val) else None

def parse_merger_overview(rcept_no: str) -> dict | None:
    url = 'https://opendart.fss.or.kr/api/document.xml'
    resp = requests.get(url, params={'crtfc_key': API_KEY, 'rcept_no': rcept_no})
    if resp.status_code != 200:
        return None
    try:
        with zipfile.ZipFile(io.BytesIO(resp.content)) as z:
            with z.open(z.namelist()[0]) as f:
                xml = f.read().decode('utf-8', errors='replace')

        # (1) 합병 개요 파싱
        part = xml.split("(1) 합병 당사회사의 개요", 1)
        if len(part) < 2:
            return None
        block = "(1) 합병 당사회사의 개요" + part[1]
        soup = BeautifulSoup(block, 'html.parser')
        table = soup.find('table')
        rows = table.find_all('tr') if table else []

        merger_data = {}
        keys = {
            '법인명': 'corp_name',
            '납입자본금(주1)': 'capital',
            '자산총액(주2)': 'assets',
            '주권상장여부': 'listing',
            '발행주식의 종류 및 수': 'shares'
        }
        for tr in rows:
            cols = tr.find_all('td')
            texts = [c.get_text(strip=True) for c in cols]
            if len(texts) == 3 and texts[0] in keys:
                merger_data[keys[texts[0]]] = {
                    '합병법인': texts[1],
                    '피합병법인': texts[2]
                }

        # 사업 개요 파싱
        starts = [m.start() for m in re.finditer(re.escape("1. 사업의 개요"), xml)]
        blocks = []
        for i, s in enumerate(starts):
            e = starts[i+1] if i+1 < len(starts) else len(xml)
            blocks.append(xml[s:e])

        end_marker = re.compile(r"(합병의 개요|2\. 주요 제품 및 서비스)")
        combined = []
        for blk in blocks:
            m = end_marker.search(blk)
            cut = blk[:m.start()] if m else blk
            sp = BeautifulSoup(cut, 'html.parser')
            paras = [p.get_text(strip=True) for p in sp.find_all('p') if p.get_text(strip=True)]
            combined.append("\n".join(paras))

        사업_합병 = combined[0] if len(combined) > 0 else ""
        사업_피합병 = combined[1] if len(combined) > 1 else ""

        def G(d, f, r): return d.get(f, {}).get(r)
        def N(d, f, r): return clean_number(G(d, f, r))

        return {
            '공시회사': G(merger_data, 'corp_name', '합병법인'),
            '피합병법인': G(merger_data, 'corp_name', '피합병법인'),
            '납입자본금(합병)': N(merger_data, 'capital', '합병법인'),
            '납입자본금(피합병)': N(merger_data, 'capital', '피합병법인'),
            '자산총액(합병)': N(merger_data, 'assets', '합병법인'),
            '자산총액(피합병)': N(merger_data, 'assets', '피합병법인'),
            '합병법인 상장': G(merger_data, 'listing', '합병법인'),
            '피합병법인 상장': G(merger_data, 'listing', '피합병법인'),
            '발행주식수(합병)': N(merger_data, 'shares', '합병법인'),
            '발행주식수(피합병)': N(merger_data, 'shares', '피합병법인'),
            '사업(합병회사)': 사업_합병,
            '사업(피합병회사)': 사업_피합병
        }
    except Exception:
        return None

def get_merger_reports():
    url = 'https://opendart.fss.or.kr/api/list.json'
    params = {
        'crtfc_key': API_KEY,
        'pblntf_detail_ty': 'C004',
        'bgn_de': '20250501',
        'end_de': '20250610',
        'last_reprt_at': 'Y',
        'page_no': 1,
        'page_count': 100
    }
    resp = requests.get(url, params=params)
    data = resp.json()
    df = pd.DataFrame(data['list'])[['rcept_dt', 'rcept_no', 'corp_name', 'report_nm']]
    df = df[df['report_nm'].str.contains(r'증권신고서\(합병', na=False)].reset_index(drop=True)

    results = []
    for _, row in tqdm(df.iterrows(), total=len(df), desc="📄 파싱 중"):
        parsed = parse_merger_overview(row['rcept_no'])
        if parsed:
            parsed['날짜'] = row['rcept_dt']
            results.append(parsed)

    return pd.DataFrame(results)

# 실행 예시
df_all = get_merger_reports()
df_all

📄 파싱 중: 100%|██████████| 3/3 [00:03<00:00,  1.01s/it]


,공시회사,피합병법인,납입자본금(합병),납입자본금(피합병),자산총액(합병),자산총액(피합병),합병법인 상장,피합병법인 상장,발행주식수(합병),발행주식수(피합병),사업(합병회사),사업(피합병회사),날짜
0,주식회사 애드포러스,신영해피투모로우제8호기업인수목적주식회사,2140680000,676000000,24881222524,15608171880,None,None,4281360,6760000,가. 용어설명\nAD\n(Advertisement)\n광고를 게재할 수 있는 지면을...,가. 합병에 관한 사항당사는「자본시장과 금융투자업에 관한 법률」에 따라 회사의 주권...,20250604
1,키움제8호기업인수목적 주식회사,주식회사 지슨,588000000,8664789500,12311598310,15106836691,상장(코스닥),상장(코넥스),5880000,17329579,당사는 자본시장과 금융투자업에 관한 법률 시행령(이하 법 시행령) 제6조 제4항 제...,당사는 2000년에 설립된 국내 최고의 전파분석 기술을 보유한 보안솔루션 기업으로서...,20250528


In [157]:
df_all.to_excel('test2.xlsx')

In [158]:
import re
import io
import zipfile
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd

API_KEY = '17db1c341b5874ad5ad8a43382af6b17d3009380'

def clean_number(val: str) -> int | None:
    return int(re.sub(r'[^\d]', '', val)) if isinstance(val, str) and re.search(r'\d', val) else None

def parse_merger_overview(rcept_no: str) -> dict | None:
    url = 'https://opendart.fss.or.kr/api/document.xml'
    resp = requests.get(url, params={'crtfc_key': API_KEY, 'rcept_no': rcept_no})
    if resp.status_code != 200:
        return None
    try:
        with zipfile.ZipFile(io.BytesIO(resp.content)) as z:
            with z.open(z.namelist()[0]) as f:
                xml = f.read().decode('utf-8', errors='replace')

        # (1) 합병 개요 파싱
        part = xml.split("(1) 합병 당사회사의 개요", 1)
        if len(part) < 2:
            return None
        block = "(1) 합병 당사회사의 개요" + part[1]
        soup = BeautifulSoup(block, 'html.parser')
        table = soup.find('table')
        rows = table.find_all('tr') if table else []

        merger_data = {}
        keys = {
            '법인명': 'corp_name',
            '납입자본금(주1)': 'capital',
            '자산총액(주2)': 'assets',
            '주권상장여부': 'listing',
            '발행주식의 종류 및 수': 'shares'
        }
        for tr in rows:
            cols = tr.find_all('td')
            texts = [c.get_text(strip=True) for c in cols]
            if len(texts) == 3 and texts[0] in keys:
                merger_data[keys[texts[0]]] = {
                    '합병법인': texts[1],
                    '피합병법인': texts[2]
                }

        # 사업 개요 파싱
        starts = [m.start() for m in re.finditer(re.escape("1. 사업의 개요"), xml)]
        blocks = []
        for i, s in enumerate(starts):
            e = starts[i+1] if i+1 < len(starts) else len(xml)
            blocks.append(xml[s:e])

        end_marker = re.compile(r"(합병의 개요|2\. 주요 제품 및 서비스)")
        combined = []
        for blk in blocks:
            # 종료 지점 자르기
            m = end_marker.search(blk)
            cut = blk[:m.start()] if m else blk

            sp = BeautifulSoup(cut, 'html.parser')
            first_para = None
            for p in sp.find_all('p'):
                text = p.get_text(strip=True)
                if text:
                    first_para = text
                    break
            combined.append(first_para or "")

        사업_합병 = combined[0]
        사업_피합병 = combined[1] if len(combined) > 1 else ""

        def G(d, f, r): return d.get(f, {}).get(r)
        def N(d, f, r): return clean_number(G(d, f, r))

        return {
            '공시회사': G(merger_data, 'corp_name', '합병법인'),
            '피합병법인': G(merger_data, 'corp_name', '피합병법인'),
            '납입자본금(합병)': N(merger_data, 'capital', '합병법인'),
            '납입자본금(피합병)': N(merger_data, 'capital', '피합병법인'),
            '자산총액(합병)': N(merger_data, 'assets', '합병법인'),
            '자산총액(피합병)': N(merger_data, 'assets', '피합병법인'),
            '합병법인 상장': G(merger_data, 'listing', '합병법인'),
            '피합병법인 상장': G(merger_data, 'listing', '피합병법인'),
            '발행주식수(합병)': N(merger_data, 'shares', '합병법인'),
            '발행주식수(피합병)': N(merger_data, 'shares', '피합병법인'),
            '사업(합병회사)': 사업_합병,
            '사업(피합병회사)': 사업_피합병
        }
    except Exception:
        return None

def get_merger_reports():
    url = 'https://opendart.fss.or.kr/api/list.json'
    params = {
        'crtfc_key': API_KEY,
        'pblntf_detail_ty': 'C004',
        'bgn_de': '20250501',
        'end_de': '20250610',
        'last_reprt_at': 'Y',
        'page_no': 1,
        'page_count': 100
    }
    resp = requests.get(url, params=params)
    data = resp.json()
    df = pd.DataFrame(data['list'])[['rcept_dt', 'rcept_no', 'corp_name', 'report_nm']]
    df = df[df['report_nm'].str.contains(r'증권신고서\(합병', na=False)].reset_index(drop=True)

    results = []
    for _, row in tqdm(df.iterrows(), total=len(df), desc="📄 파싱 중"):
        parsed = parse_merger_overview(row['rcept_no'])
        if parsed:
            parsed['날짜'] = row['rcept_dt']
            results.append(parsed)

    return pd.DataFrame(results)

# 실행 예시
df_all = get_merger_reports()
df_all

📄 파싱 중: 100%|██████████| 3/3 [00:02<00:00,  1.19it/s]


,공시회사,피합병법인,납입자본금(합병),납입자본금(피합병),자산총액(합병),자산총액(피합병),합병법인 상장,피합병법인 상장,발행주식수(합병),발행주식수(피합병),사업(합병회사),사업(피합병회사),날짜
0,주식회사 애드포러스,신영해피투모로우제8호기업인수목적주식회사,2140680000,676000000,24881222524,15608171880,None,None,4281360,6760000,가. 용어설명,가. 합병에 관한 사항당사는「자본시장과 금융투자업에 관한 법률」에 따라 회사의 주권...,20250604
1,키움제8호기업인수목적 주식회사,주식회사 지슨,588000000,8664789500,12311598310,15106836691,상장(코스닥),상장(코넥스),5880000,17329579,당사는 자본시장과 금융투자업에 관한 법률 시행령(이하 법 시행령) 제6조 제4항 제...,당사는 2000년에 설립된 국내 최고의 전파분석 기술을 보유한 보안솔루션 기업으로서...,20250528


In [159]:
df_all.to_excel('test2.xlsx')

# 최최최종

### 사업개요 전처리 x

In [195]:
import re
import io
import zipfile
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd

API_KEY = '17db1c341b5874ad5ad8a43382af6b17d3009380'

def clean_number(val: str) -> int | None:
    return int(re.sub(r'[^\d]', '', val)) if isinstance(val, str) and re.search(r'\d', val) else None

def parse_merger_overview(rcept_no: str) -> dict | None:
    # 1) Download & unzip
    url = 'https://opendart.fss.or.kr/api/document.xml'
    resp = requests.get(url, params={'crtfc_key': API_KEY, 'rcept_no': rcept_no})
    resp.raise_for_status()
    xml = None
    with zipfile.ZipFile(io.BytesIO(resp.content)) as z:
        with z.open(z.namelist()[0]) as f:
            xml = f.read().decode('utf-8', errors='replace')

    # 2) Merge overview parsing
    part = xml.split("(1) 합병 당사회사의 개요", 1)
    if len(part) < 2:
        return None
    block = "(1) 합병 당사회사의 개요" + part[1]
    soup = BeautifulSoup(block, 'html.parser')
    table = soup.find('table')
    rows = table.find_all('tr') if table else []

    merger_data = {}
    keys = {
        '법인명': 'corp_name',
        '납입자본금(주1)': 'capital',
        '자산총액(주2)': 'assets',
        '주권상장여부': 'listing',
        '발행주식의 종류 및 수': 'shares'
    }
    for tr in rows:
        cols = tr.find_all('td')
        texts = [c.get_text(strip=True) for c in cols]
        if len(texts) == 3 and texts[0] in keys:
            merger_data[keys[texts[0]]] = {
                '합병법인': texts[1],
                '피합병법인': texts[2]
            }

    # 3) Business overview parsing
    starts = [m.start() for m in re.finditer(re.escape("1. 사업의 개요"), xml)]
    blocks = []
    for i, s in enumerate(starts):
        e = starts[i+1] if i+1 < len(starts) else len(xml)
        blocks.append(xml[s:e])

    end_marker = re.compile(
       r"(?:1\. 합병의 개요|가\. 합병의 개요|나\. 합병의 개요|2\. 주요 제품 및 서비스)"
    )
    combined_paras = []

    for blk in blocks:
        if "나. 회사의 현황" in blk:
            # after '회사의 현황', first <p>
            sub = blk.split("나. 회사의 현황", 1)[1]
            soup2 = BeautifulSoup(sub, 'html.parser')
            para = ""
            for p in soup2.find_all('p'):
                txt = p.get_text(strip=True)
                if txt:
                    para = txt
                    break
            combined_paras.append(para)
        else:
            # until end_marker, then take only the first <p>
            m = end_marker.search(blk)
            cut = blk[:m.start()] if m else blk
            soup2 = BeautifulSoup(cut, 'html.parser')
            first_para = ""
            for p in soup2.find_all('p'):
                txt = p.get_text(strip=True)
                if txt:
                    first_para = txt
                    break
            combined_paras.append(first_para)

    business_merge = combined_paras[0] if len(combined_paras) > 0 else ""
    business_target = combined_paras[1] if len(combined_paras) > 1 else ""

    # 4) Build result
    def G(d, f, r): return d.get(f, {}).get(r)
    def N(d, f, r): return clean_number(G(d, f, r))

    return {
        '공시회사':            G(merger_data, 'corp_name', '합병법인'),
        '피합병법인':          G(merger_data, 'corp_name', '피합병법인'),
        '납입자본금(합병)':    N(merger_data, 'capital', '합병법인'),
        '납입자본금(피합병)':  N(merger_data, 'capital', '피합병법인'),
        '자산총액(합병)':      N(merger_data, 'assets', '합병법인'),
        '자산총액(피합병)':    N(merger_data, 'assets', '피합병법인'),
        '합병법인 상장':       G(merger_data, 'listing', '합병법인'),
        '피합병법인 상장':     G(merger_data, 'listing', '피합병법인'),
        '발행주식수(합병)':    N(merger_data, 'shares', '합병법인'),
        '발행주식수(피합병)':  N(merger_data, 'shares', '피합병법인'),
        '사업개요(합병)':      business_merge,
        '사업개요(피합병)':    business_target
    }

def get_merger_reports():
    url = 'https://opendart.fss.or.kr/api/list.json'
    params = {
        'crtfc_key': API_KEY,
        'pblntf_detail_ty': 'C004',
        'bgn_de': '20250101',
        'end_de': '20250310',
        'last_reprt_at': 'Y',
        'page_no': 1,
        'page_count': 100
    }
    resp = requests.get(url, params=params)
    data = resp.json()
    df = pd.DataFrame(data['list'])[['rcept_dt', 'rcept_no', 'corp_name', 'report_nm']]
    df = df[
        df['report_nm'].str.contains(r'증권신고서\(합병', na=False) &
        ~df['report_nm'].str.contains('첨부정정', na=False)
    ].reset_index(drop=True)

    results = []
    for _, row in tqdm(df.iterrows(), total=len(df), desc="📄 파싱 중"):
        parsed = parse_merger_overview(row['rcept_no'])
        if parsed:
            parsed['날짜'] = row['rcept_dt']
            results.append(parsed)

    return pd.DataFrame(results)

# Execute example
df_all = get_merger_reports()
df_all

📄 파싱 중: 100%|██████████| 4/4 [00:05<00:00,  1.28s/it]


,공시회사,피합병법인,납입자본금(합병),납입자본금(피합병),자산총액(합병),자산총액(피합병),합병법인 상장,피합병법인 상장,발행주식수(합병),발행주식수(피합병),사업개요(합병),사업개요(피합병),날짜
0,주식회사 바이오포트코리아,디비금융제11호기업인수목적 주식회사,727788000,531500000,50339983452,12252217006,None,None,3638940.0,5315000.0,"당사는 스낵류, 액상차, 음료 등 식품 및 음료 제조, 판매 사업을 영위하고 있습니...",1. 합병에 관한 사항,20250225
1,우양에이치씨 주식회사,케이비제26호기업인수목적 주식회사,6827817000,550500000,261045455458,12131001817,None,None,13655634.0,5505000.0,"당사는 지난 30년간 화공, 에너지 등 플랜트산업 전반에 걸쳐 세계 곳곳에 다양한 ...","당사는 「자본시장과 금융투자업에 관한 법률」(이하 ""법""이라 한다) 제373조에 따...",20250205
2,주식회사 에이아이코리아,한국제14호기업인수목적 주식회사,3462419500,481000000,72224746632,9422173860,None,None,NaN,NaN,당사는 2차전지 생산설비의 전해액 중앙공급시스템사업(CESS)을 주요 사업으로 영위...,"1-1. 합병에 관한 사항당사는 「자본시장과 금융투자업에 관한 법률」(이하 ""법""이...",20250110


### 사업개요 전처리 o

In [228]:
import re
import io
import zipfile
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd

API_KEY = '17db1c341b5874ad5ad8a43382af6b17d3009380'

def clean_number(val: str) -> int | None:
    if not isinstance(val, str):
        return None
    s = re.sub(r'\(.*?\)', '', val)
    digits = re.sub(r'[^\d]', '', s)
    return int(digits) if digits else None

def parse_amount(val: str) -> int | None:
    if not isinstance(val, str):
        return None
    s = re.sub(r'\(.*?\)', '', val)
    digits = re.sub(r'[^\d]', '', s)
    if not digits:
        return None
    base = int(digits)
    if '백만' in s or '백' in s:
        return base * 1_000_000
    return base

def parse_merger_overview(rcept_no: str) -> dict | None:
    resp = requests.get(
        'https://opendart.fss.or.kr/api/document.xml',
        params={'crtfc_key': API_KEY, 'rcept_no': rcept_no}
    )
    resp.raise_for_status()
    with zipfile.ZipFile(io.BytesIO(resp.content)) as z:
        with z.open(z.namelist()[0]) as f:
            xml = f.read().decode('utf-8', errors='replace')

    part = xml.split("(1) 합병 당사회사의 개요", 1)
    if len(part) < 2:
        return None

    block = "(1) 합병 당사회사의 개요" + part[1]
    soup = BeautifulSoup(block, 'html.parser')
    table = soup.find('table')
    rows = table.find_all('tr') if table else []

    merger_data = {}
    mapping = {
        r'법인명':             'corp_name',
        r'납입자본금':         'capital',
        r'자산총액':           'assets',
        r'주권상장여부':        'listing',
        r'발행주식.*수':       'shares',
    }
    for tr in rows:
        cols = tr.find_all('td')
        texts = [c.get_text(strip=True) for c in cols]
        if len(texts) != 3:
            continue
        label = texts[0]
        for pat, field in mapping.items():
            if re.search(pat, label):
                merger_data[field] = {
                    '합병법인':   texts[1],
                    '피합병법인': texts[2]
                }
                break

    starts = [m.start() for m in re.finditer(re.escape("1. 사업의 개요"), xml)]
    blocks = [xml[starts[i]: (starts[i+1] if i+1 < len(starts) else len(xml))] for i in range(len(starts))]

    end_marker = re.compile(r"(?:1\. 합병의 개요|가\. 합병의 개요|나\. 합병의 개요|2\. 주요 제품 및 서비스)")
    combined = []
    for blk in blocks:
        if "나. 회사의 현황" in blk:
            sub = blk.split("나. 회사의 현황", 1)[1]
            soup2 = BeautifulSoup(sub, 'html.parser')
            para = next((p.get_text(strip=True) for p in soup2.find_all('p') if p.get_text(strip=True)), "")
            combined.append(para)
        else:
            m = end_marker.search(blk)
            cut = blk[:m.start()] if m else blk
            soup2 = BeautifulSoup(cut, 'html.parser')
            para = next((p.get_text(strip=True) for p in soup2.find_all('p') if p.get_text(strip=True)), "")
            combined.append(para)

    business_merge = combined[0] if combined else ""
    business_target = combined[1] if len(combined) > 1 else ""

    def G(d, f, r): return d.get(f, {}).get(r)
    def N(d, f, r): return parse_amount(G(d, f, r))

    return {
        '공시회사':           G(merger_data, 'corp_name', '합병법인'),
        '피합병법인':         G(merger_data, 'corp_name', '피합병법인'),
        '납입자본금(합병)':   N(merger_data, 'capital', '합병법인'),
        '납입자본금(피합병)': N(merger_data, 'capital', '피합병법인'),
        '자산총액(합병)':     N(merger_data, 'assets', '합병법인'),
        '자산총액(피합병)':   N(merger_data, 'assets', '피합병법인'),
        '합병법인 상장':      G(merger_data, 'listing', '합병법인'),
        '피합병법인 상장':    G(merger_data, 'listing', '피합병법인'),
        '발행주식수(합병)':   clean_number(G(merger_data, 'shares', '합병법인')),
        '발행주식수(피합병)': clean_number(G(merger_data, 'shares', '피합병법인')),
        '사업개요(합병)':     business_merge,
        '사업개요(피합병)':   business_target
    }



def get_merger_reports():
    url = 'https://opendart.fss.or.kr/api/list.json'
    params = {
        'crtfc_key': API_KEY,
        'pblntf_detail_ty': 'C004',
        'bgn_de': '20250401',
        'end_de': '20250624',
        'last_reprt_at': 'Y',
        'page_no': 1,
        'page_count': 100
    }
    resp = requests.get(url, params=params)
    data = resp.json()
    df = pd.DataFrame(data['list'])[['rcept_dt', 'rcept_no', 'corp_name', 'report_nm']]
    df = df[
        df['report_nm'].str.contains(r'증권신고서\(합병', na=False) &
        ~df['report_nm'].str.contains('첨부정정', na=False)
    ].reset_index(drop=True)

    results = []
    for _, row in tqdm(df.iterrows(), total=len(df), desc="📄 파싱 중"):
        parsed = parse_merger_overview(row['rcept_no'])
        if parsed:
            parsed['날짜'] = row['rcept_dt']
            results.append(parsed)

    # 최종 DataFrame 생성
    df_all = pd.DataFrame(results)
    # 사업개요 컬럼 후처리: SPAC/스팩/인수목적 포함 시 피합병, 아니면 합병 개요 사용
    df_all['사업개요'] = df_all.apply(
        lambda row: row['사업개요(피합병)']
        if re.search(r'인수목적|스팩|SPAC', row['공시회사'])
        else row['사업개요(합병)'],
        axis=1
    )
    # 불필요 컬럼 삭제
    df_all = df_all.drop(columns=['사업개요(합병)', '사업개요(피합병)'])
    cols = ['공시회사', '피합병법인', '날짜', '납입자본금(합병)', '납입자본금(피합병)', '자산총액(합병)', '자산총액(피합병)',
            '합병법인 상장', '피합병법인 상장', '발행주식수(합병)', '발행주식수(피합병)', '사업개요'
            ]
    df_all = df_all[cols]
    df_all = df_all.sort_values(by='날짜')
    return df_all


# 실행 예시
df_all = get_merger_reports()
df_all

📄 파싱 중: 100%|██████████| 7/7 [00:08<00:00,  1.18s/it]


,공시회사,피합병법인,날짜,납입자본금(합병),납입자본금(피합병),자산총액(합병),자산총액(피합병),합병법인 상장,피합병법인 상장,발행주식수(합병),발행주식수(피합병),사업개요
4,주식회사 뉴키즈온,케이비제28호기업인수목적 주식회사,20250407,682834600,550500000,45726999119,12166140945,None,None,6988346,5505000,"당사는 유아동 의류 및 화장품을 제조, 판매하는 회사로 차별화된 디자인과 합리적인 ..."
3,케이지에이 주식회사,삼성기업인수목적9호 주식회사,20250410,4602000000,1105000000,72707000000,25413000000,None,None,9204195,11050000,"당사는 이차전지 제조공정 중 전극공정 장비(코터, 프레스, 슬리터) 개발 및 제조를..."
2,키움제8호기업인수목적 주식회사,주식회사 지슨,20250528,588000000,8664789500,12311598310,15106836691,상장(코스닥),상장(코넥스),5880000,17329579,당사는 2000년에 설립된 국내 최고의 전파분석 기술을 보유한 보안솔루션 기업으로서...
1,주식회사 애드포러스,신영해피투모로우제8호기업인수목적주식회사,20250604,2140680000,676000000,24881222524,15608171880,None,None,4281360,6760000,당사는 온라인 광고산업에서 광고주와 광고매체를 연결하는 모바일 광고 플랫폼을 주요 ...
0,오아 주식회사,미래에셋비전기업인수목적2호 주식회사,20250617,3262700000,495000000,69207767055,12516196755,None,None,4637372,4950000,"당사는 중소형가전과 건강푸드를 전문으로 취급하는 종합 소비재 기업으로, '오아(OA..."


### 엑셀 저장

In [227]:
from datetime import datetime
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Font
from openpyxl.utils import get_column_letter
import pandas as pd

# (1) 데이터프레임 준비
# df_all = get_merger_reports()

path = "국내 주요 공시 정리.xlsx"

# (2) 기존 워크북 열고 '합병' 시트만 삭제
wb = load_workbook(path)
if "합병" in wb.sheetnames:
    wb.remove(wb["합병"])
wb.save(path)

# (3) 새로 '합병' 시트에 쓰기 (나머지 시트는 그대로 유지)
with pd.ExcelWriter(path, engine="openpyxl", mode="a", if_sheet_exists="new") as writer:
    df_all.to_excel(writer, index=False, sheet_name="합병")
    ws = writer.sheets["합병"]

    header     = [cell.value for cell in ws[1]]
    date_cols  = {"날짜"}
    comma_cols = {
        "납입자본금(합병)", "납입자본금(피합병)",
        "자산총액(합병)",   "자산총액(피합병)",
        "발행주식수(합병)", "발행주식수(피합병)"
    }
    wrap_col = "사업개요"

    # (4) 데이터 영역 포맷
    for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=len(header)):
        for cell in row:
            col = header[cell.column-1]

            if col in date_cols and isinstance(cell.value, str):
                try:
                    cell.value = datetime.strptime(cell.value, "%Y%m%d")
                except ValueError:
                    pass
                cell.number_format = "yyyy-mm-dd"

            elif col in comma_cols and isinstance(cell.value, (int, float)):
                cell.number_format = "#,##0"

            if col == wrap_col:
                cell.alignment = Alignment(wrap_text=True)
            else:
                cell.alignment = Alignment(horizontal="center", vertical="center")

            cell.font = Font(size=10)

    # (5) 헤더 포맷
    for cell in ws[1]:
        cell.alignment = Alignment(horizontal="center", vertical="center")
        cell.font = Font(size=10, bold=True)

    # (6) 열 너비 자동 조정
    for idx, _ in enumerate(header, start=1):
        letter = get_column_letter(idx)
        max_length = max(
            len(str(c.value)) if c.value is not None else 0
            for c in ws[letter]
        )
        ws.column_dimensions[letter].width = max_length + 2

print("✅ '합병' 시트만 교체하고, 나머지 시트는 그대로 유지했습니다.")

✅ '합병' 시트만 교체하고, 나머지 시트는 그대로 유지했습니다.


In [ ]:
from datetime import datetime
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Font
from openpyxl.utils import get_column_letter
import pandas as pd

# ———(1) 준비— df_all = get_merger_reports() 등으로 만드신 DataFrame
# 예시:
# df_all = get_merger_reports()

path = "국내 주요 공시 정리.xlsx"
wb   = load_workbook(path)
ws   = wb["합병"]  # 반드시 기존에 "합병" 시트가 있어야 합니다

# ———(2) 워크시트 헤더(1행)와 df_all.columns 매핑
header = [cell.value for cell in ws[1]]
col_map = {col: idx for idx, col in enumerate(df_all.columns)}  # df_all.columns 순서대로

# ———(3) append 시작 행 계산
start_row = ws.max_row + 1

# ———(4) df_all을 append
for ridx, row in enumerate(df_all.itertuples(index=False), start=start_row):
    for cidx, col_name in enumerate(header, start=1):
        if col_name in col_map:
            value = row[col_map[col_name]]
            ws.cell(row=ridx, column=cidx, value=value)

# ———(5) 추가된 영역에만 포맷 적용
date_cols  = {"날짜"}
comma_cols = {
    "납입자본금(합병)", "납입자본금(피합병)",
    "자산총액(합병)",   "자산총액(피합병)",
    "발행주식수(합병)", "발행주식수(피합병)"
}
wrap_col = "사업개요"

for ridx in range(start_row, start_row + len(df_all)):
    for cidx, col_name in enumerate(header, start=1):
        cell = ws.cell(row=ridx, column=cidx)
        # 날짜 문자열 → datetime
        if col_name in date_cols and isinstance(cell.value, str):
            try:
                cell.value = datetime.strptime(cell.value, "%Y%m%d")
            except ValueError:
                pass
            cell.number_format = "yyyy-mm-dd"
        # 천 단위 콤마
        elif col_name in comma_cols and isinstance(cell.value, (int, float)):
            cell.number_format = "#,##0"
        # 줄바꿈/중앙정렬 & 폰트
        if col_name == wrap_col:
            cell.alignment = Alignment(wrap_text=True)
        else:
            cell.alignment = Alignment(horizontal="center", vertical="center")
        cell.font = Font(size=10)

# ———(7) 저장
wb.save(path)
print("✅ df_all이 '합병' 시트 아래에 append 되었고, 나머지 시트는 그대로 유지됩니다.")

✅ df_all이 '합병' 시트 아래에 append 되었고, 나머지 시트는 그대로 유지됩니다.


# 전체 플로우

## 데이터 추출 및 전처리

In [36]:
import re
import io
import zipfile
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd

API_KEY = '17db1c341b5874ad5ad8a43382af6b17d3009380'

def clean_number(val: str) -> int | None:
    if not isinstance(val, str):
        return None
    s = re.sub(r'\(.*?\)', '', val)
    digits = re.sub(r'[^\d]', '', s)
    return int(digits) if digits else None

def parse_amount(val: str) -> int | None:
    if not isinstance(val, str):
        return None
    s = re.sub(r'\(.*?\)', '', val)
    digits = re.sub(r'[^\d]', '', s)
    if not digits:
        return None
    base = int(digits)
    if '백만' in s or '백' in s:
        return base * 1_000_000
    return base

def parse_merger_overview(rcept_no: str, corp_name: str) -> dict | None:
    resp = requests.get(
        'https://opendart.fss.or.kr/api/document.xml',
        params={'crtfc_key': API_KEY, 'rcept_no': rcept_no}
    )
    resp.raise_for_status()
    with zipfile.ZipFile(io.BytesIO(resp.content)) as z:
        with z.open(z.namelist()[0]) as f:
            xml = f.read().decode('utf-8', errors='replace')

    part = xml.split("(1) 합병 당사회사의 개요", 1)
    if len(part) < 2:
        return None

    block = "(1) 합병 당사회사의 개요" + part[1]
    soup = BeautifulSoup(block, 'html.parser')
    table = soup.find('table')
    rows = table.find_all('tr') if table else []

    merger_data = {}
    mapping = {
        r'법인명':             'corp_name',
        r'납입자본금':         'capital',
        r'자산총액':           'assets',
        r'주권상장여부':        'listing',
        r'발행주식.*수':       'shares',
    }
    for tr in rows:
        cols = tr.find_all('td')
        texts = [c.get_text(strip=True) for c in cols]
        if len(texts) != 3:
            continue
        label = texts[0]
        for pat, field in mapping.items():
            if re.search(pat, label):
                merger_data[field] = {
                    '합병법인':   texts[1],
                    '피합병법인': texts[2]
                }
                break

    starts = [m.start() for m in re.finditer(re.escape("1. 사업의 개요"), xml)]
    blocks = [xml[starts[i]: (starts[i+1] if i+1 < len(starts) else len(xml))] for i in range(len(starts))]

    end_marker = re.compile(r"(?:1\. 합병의 개요|가\. 합병의 개요|나\. 합병의 개요|2\. 주요 제품 및 서비스)")

    combined = []
    for blk in blocks:
        if "나. 회사의 현황" in blk:
            sub = blk.split("나. 회사의 현황", 1)[1]
            soup2 = BeautifulSoup(sub, 'html.parser')
            para = next((p.get_text(strip=True) for p in soup2.find_all('p') if p.get_text(strip=True)), "")
            combined.append(para)
        else:
            m = end_marker.search(blk)
            cut = blk[:m.start()] if m else blk
            soup2 = BeautifulSoup(cut, 'html.parser')
            para = next((p.get_text(strip=True) for p in soup2.find_all('p') if p.get_text(strip=True)), "")
            combined.append(para)

    business_merge = combined[0] if combined else ""
    business_target = combined[1] if len(combined) > 1 else ""
    
    def G(d, f, r): return d.get(f, {}).get(r)
    def N(d, f, r): return parse_amount(G(d, f, r))

    return {
        '공시회사': corp_name,
        '합병법인':           G(merger_data, 'corp_name', '합병법인'),
        '피합병법인':         G(merger_data, 'corp_name', '피합병법인'),
        '납입자본금(합병)':   N(merger_data, 'capital', '합병법인'),
        '납입자본금(피합병)': N(merger_data, 'capital', '피합병법인'),
        '자산총액(합병)':     N(merger_data, 'assets', '합병법인'),
        '자산총액(피합병)':   N(merger_data, 'assets', '피합병법인'),
        '합병법인 상장':      G(merger_data, 'listing', '합병법인'),
        '피합병법인 상장':    G(merger_data, 'listing', '피합병법인'),
        '발행주식수(합병)':   clean_number(G(merger_data, 'shares', '합병법인')),
        '발행주식수(피합병)': clean_number(G(merger_data, 'shares', '피합병법인')),
        '사업개요(합병)':     business_merge,
        '사업개요(피합병)':   business_target
    }



def get_merger_reports():
    url = 'https://opendart.fss.or.kr/api/list.json'
    params = {
        'crtfc_key': API_KEY,
        'pblntf_detail_ty': 'C004',
        'bgn_de': '20241001',
        'end_de': '20241231',
        'last_reprt_at': 'Y',
        'page_no': 1,
        'page_count': 100
    }
    resp = requests.get(url, params=params)
    data = resp.json()
    df = pd.DataFrame(data['list'])[['rcept_dt', 'rcept_no', 'corp_name', 'report_nm']]
    df = df[
        df['report_nm'].str.contains(r'증권신고서\(합병', na=False) &
        ~df['report_nm'].str.contains('첨부정정', na=False)
    ].reset_index(drop=True)

    results = []
    for _, row in tqdm(df.iterrows(), total=len(df), desc="📄 파싱 중"):
        parsed = parse_merger_overview(row['rcept_no'], row['corp_name'])
        if parsed:
            parsed['최종보고일'] = row['rcept_dt']
            results.append(parsed)

    df_all = pd.DataFrame(results)

    df_all['사업개요'] = df_all.apply(
        lambda row: row['사업개요(피합병)']
        if re.search(r'인수목적|스팩|SPAC', row['합병법인'])
        else row['사업개요(합병)'],
        axis=1
    )

    df_all = df_all.drop(columns=['사업개요(합병)', '사업개요(피합병)'])
    cols = ['공시회사', '합병법인', '피합병법인', '최종보고일', '납입자본금(합병)', '납입자본금(피합병)', '자산총액(합병)', '자산총액(피합병)',
            '합병법인 상장', '피합병법인 상장', '발행주식수(합병)', '발행주식수(피합병)', '사업개요'
            ]
    df_all = df_all[cols]
    df_all = df_all.sort_values(by='최종보고일').reset_index(drop=True)
    return df_all


df_all = get_merger_reports()
df_all

📄 파싱 중: 100%|██████████| 8/8 [00:07<00:00,  1.12it/s]


,공시회사,합병법인,피합병법인,최종보고일,납입자본금(합병),납입자본금(피합병),자산총액(합병),자산총액(피합병),합병법인 상장,피합병법인 상장,발행주식수(합병),발행주식수(피합병),사업개요
0,엠에프씨,엠에프씨 주식회사,하나금융21호기업인수목적 주식회사,20241010,3441692500,741000000,27990800525,15222482341,비상장,상장,6883385,7410000,"당사는 원료의약품/소재개발 및 제조 전문 회사로, 유기합성 및 대량생산 기술, 신규..."
1,에스지헬스케어,주식회사 에스지헬스케어,하나금융22호기업인수목적 주식회사,20241030,910000000,601000000,27473106101,12617294250,비상장,상장,9100000,6010000,당사는 2009년 07월 17일 설립되어 현재 의료용 영상진단기기 제조 및 판매를 ...
2,블랙야크아이앤씨,주식회사 블랙야크아이앤씨,미래에셋비전기업인수목적1호 주식회사,20241106,2110000000,618000000,33000000000,15344000000,None,None,21100000,6176000,"당사는 산업안전용품산업 또는 재난안전산업과 관련하여 안전화, 산업안전용품 및 안전복..."
3,에르코스,주식회사 에르코스 농업회사법인,키움제6호기업인수목적 주식회사,20241209,3137995000,343000000,80230663902,7970165712,비상장,상장,6275990,3430000,
4,에스엠씨지,주식회사 에스엠씨지,키움제7호기업인수목적 주식회사,20241217,1575491800,410500000,62561313007,9767169402,비상장,상장,15754918,4105000,합병법인은 유리용기 기반의 화장품용 용기 전체 패키지(Full package)를 제...


## 엑셀로 저장

In [235]:
from datetime import datetime
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Font
import pandas as pd

path = "국내 주요 공시 정리.xlsx"
wb   = load_workbook(path)
ws   = wb["합병"] 

header = [cell.value for cell in ws[1]]
col_map = {col: idx for idx, col in enumerate(df_all.columns)} 

start_row = ws.max_row + 1

for ridx, row in enumerate(df_all.itertuples(index=False), start=start_row):
    for cidx, col_name in enumerate(header, start=1):
        if col_name in col_map:
            value = row[col_map[col_name]]
            ws.cell(row=ridx, column=cidx, value=value)

date_cols  = {"최종보고일"}
comma_cols = {
    "납입자본금(합병)", "납입자본금(피합병)",
    "자산총액(합병)",   "자산총액(피합병)",
    "발행주식수(합병)", "발행주식수(피합병)"
}
wrap_col = "사업개요"

for ridx in range(start_row, start_row + len(df_all)):
    for cidx, col_name in enumerate(header, start=1):
        cell = ws.cell(row=ridx, column=cidx)
        if col_name in date_cols and isinstance(cell.value, str):
            try:
                cell.value = datetime.strptime(cell.value, "%Y%m%d")
            except ValueError:
                pass
            cell.number_format = "yyyy-mm-dd"
        elif col_name in comma_cols and isinstance(cell.value, (int, float)):
            cell.number_format = "#,##0"
        if col_name == wrap_col:
            cell.alignment = Alignment(wrap_text=True)
        else:
            cell.alignment = Alignment(horizontal="center", vertical="center")
        cell.font = Font(size=10)

wb.save(path)
print("✅ 저장 완료")

✅ 저장 완료


In [239]:
from datetime import datetime
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Font
from openpyxl.utils import get_column_letter
import pandas as pd

path = "국내 주요 공시 정리.xlsx"
wb   = load_workbook(path)
ws   = wb["합병"]

# 1) 워크시트 헤더
header = [cell.value for cell in ws[1]]

# 2) df_all 컬럼 → 인덱스 매핑
col_map = {col: idx for idx, col in enumerate(df_all.columns)}

# 3) 기존 키→행 번호
existing = {}
공시_idx = header.index("공시회사")
피합_idx = header.index("피합병법인")
for row in ws.iter_rows(min_row=2, max_row=ws.max_row, values_only=False):
    key = (row[공시_idx].value, row[피합_idx].value)
    existing[key] = row[0].row

# 4) 포맷 대상 정의
date_cols  = {"최종보고일"}
comma_cols = {
    "납입자본금(합병)", "납입자본금(피합병)",
    "자산총액(합병)",   "자산총액(피합병)",
    "발행주식수(합병)", "발행주식수(피합병)"
}
wrap_col = "사업개요"

def apply_format(cell, col_name):
    if col_name in date_cols and isinstance(cell.value, str):
        try:
            cell.value = datetime.strptime(cell.value, "%Y%m%d")
            cell.number_format = "yyyy-mm-dd"
        except ValueError:
            pass
    elif col_name in comma_cols and isinstance(cell.value, (int, float)):
        cell.number_format = "#,##0"

    if col_name == wrap_col:
        cell.alignment = Alignment(wrap_text=True)
    else:
        cell.alignment = Alignment(horizontal="center", vertical="center")
    cell.font = Font(size=10)

# 5) append or update
start_row = ws.max_row + 1
for record in df_all.itertuples(index=False):
    key = (getattr(record, "공시회사"), getattr(record, "피합병법인"))
    if key in existing:
        r = existing[key]
        for cidx, col_name in enumerate(header, start=1):
            if col_name not in col_map:
                continue
            new_val = record[col_map[col_name]]
            cell    = ws.cell(row=r, column=cidx)
            if cell.value != new_val:
                cell.value = new_val
                apply_format(cell, col_name)
    else:
        r = start_row
        for cidx, col_name in enumerate(header, start=1):
            if col_name not in col_map:
                continue
            val  = record[col_map[col_name]]
            cell = ws.cell(row=r, column=cidx, value=val)
            apply_format(cell, col_name)
        start_row += 1

wb.save(path)
print("✅ ‘합병’ 시트에 변경사항 반영 완료. 다른 시트는 그대로 유지됩니다.")

✅ ‘합병’ 시트에 변경사항 반영 완료. 다른 시트는 그대로 유지됩니다.


In [25]:
from datetime import datetime
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Font
import pandas as pd

# 0) df_all 준비
# df_all = get_merger_reports()
# 날짜 컬럼 이름 바꾸기

path = "국내 주요 공시 정리.xlsx"
wb   = load_workbook(path)
ws   = wb["합병"] 

# 1) 워크시트 헤더
header = [cell.value for cell in ws[1]]

# ⇒ 1-A) '최초보고일' 컬럼이 없으면 '최종보고일' 오른쪽에 삽입
if "최초보고일" not in header:
    idx = header.index("최종보고일") + 1  # _0-based_ 헤더 리스트 인덱스
    # 시트에 삽입
    ws.insert_cols(idx+1)
    ws.cell(row=1, column=idx+1, value="최초보고일")
    # 파이썬 리스트에도 반영
    header.insert(idx, "최초보고일")

# 2) df_all 컬럼 → 인덱스 매핑 (append/update 시 딕셔너리 대신 위치 참조)
col_map = {col: i for i, col in enumerate(df_all.columns)}

# 3) 기존 키→행 번호
existing = {}
공시_idx = header.index("합병법인")
피합_idx = header.index("피합병법인")
for row in ws.iter_rows(min_row=2, max_row=ws.max_row, values_only=False):
    key = (row[공시_idx].value, row[피합_idx].value)
    existing[key] = row[0].row

# 4) 포맷 함수 & 대상
date_cols  = {"최종보고일","최초보고일"}
comma_cols = {
    "납입자본금(합병)", "납입자본금(피합병)",
    "자산총액(합병)",   "자산총액(피합병)",
    "발행주식수(합병)", "발행주식수(피합병)"
}
wrap_col = "사업개요"

def apply_format(cell, col_name):
    if col_name in date_cols and isinstance(cell.value, str):
        try:
            cell.value = datetime.strptime(cell.value, "%Y%m%d")
            cell.number_format = "yyyy-mm-dd"
        except ValueError:
            pass
    elif col_name in comma_cols and isinstance(cell.value, (int, float)):
        cell.number_format = "#,##0"
    if col_name == wrap_col:
        cell.alignment = Alignment(wrap_text=True)
    else:
        cell.alignment = Alignment(horizontal="center", vertical="center")
    cell.font = Font(size=10)

# 5) append or update
start_row = ws.max_row + 1
for rec in df_all.itertuples(index=False):
    key = (getattr(rec, "합병법인"), getattr(rec, "피합병법인"))
    if key in existing:
        # ▶ update only
        r = existing[key]
        for col_idx, col_name in enumerate(header, start=1):
            if col_name in col_map:
                val = rec[col_map[col_name]]
                cell = ws.cell(row=r, column=col_idx)
                if cell.value != val:
                    cell.value = val
                    apply_format(cell, col_name)
        # 최초보고일은 건드리지 않음
    else:
        # ▶ append new row
        r = start_row
        for col_idx, col_name in enumerate(header, start=1):
            if col_name in col_map:
                val = rec[col_map[col_name]]
            elif col_name == "최초보고일":
                # 신규일 때만 최종보고일 값을 복사
                val = getattr(rec, "최종보고일")
            else:
                continue
            cell = ws.cell(row=r, column=col_idx, value=val)
            apply_format(cell, col_name)
        start_row += 1

wb.save(path)
print("✅ ‘합병’ 시트에 최초보고일까지 채워넣고, 나머지 시트는 그대로 유지했습니다.")

✅ ‘합병’ 시트에 최초보고일까지 채워넣고, 나머지 시트는 그대로 유지했습니다.
